<a href="https://colab.research.google.com/github/Bhaskat/FMML_COURSE_ASSIGNMENTS/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


#Experiments with splits

1.Answer

The size of the validation set, as a percentage of the total dataset, can impact the accuracy estimation and the training process in machine learning. Here's how it's affected when you increase or reduce the percentage of the validation set:

Increase the Percentage of Validation Set:

Pros:

A larger validation set provides a more reliable estimate of how well your model is likely to perform on unseen data. It reduces the variance in your performance estimate, making it more stable.
With a larger validation set, you have more data to tune hyperparameters, which can lead to better model performance.
Cons:

If you allocate a significant portion of your data to the validation set, you have fewer examples left for training. This can lead to a model that doesn't generalize well to the test or real-world data if your training set is too small.
Reduce the Percentage of Validation Set:

Pros:

More data is available for training, which can lead to a model that learns better from the data.
Faster training time since you are using a larger portion of the data for training.
Cons:

A smaller validation set may provide a less stable estimate of your model's performance. The evaluation might be more sensitive to variations in the data, leading to less reliable results.
You may have fewer examples for hyperparameter tuning, which can make it challenging to find the best model configuration.
The choice of the percentage of the validation set should be made based on the specific characteristics of your dataset and the goals of your machine learning project. It's often a trade-off between having a more accurate estimate of performance (larger validation set) and having more data for training (smaller validation set). Cross-validation techniques, such as k-fold cross-validation, can also be used to mitigate some of these trade-offs by repeatedly splitting the data into training and validation subsets, providing a more comprehensive performance estimate.






2.Answer

The accuracy of the validation set can be affected when you increase or reduce the percentage of data allocated to the validation set in the following ways:

Increase the Percentage of Validation Set:

Pros:

More Reliable Estimate: With a larger validation set, you get a more reliable estimate of your model's performance because you are evaluating it on a larger and more representative portion of the data. This estimate is less likely to be influenced by random fluctuations in the validation set.
Cons:

Less Data for Training: Allocating a larger percentage of the data to the validation set means you have less data available for training. This can lead to a model that may not perform optimally because it has fewer examples to learn from. If the training set becomes too small, your model may not generalize well to new, unseen data.
Reduce the Percentage of Validation Set:

Pros:

More Data for Training: By reducing the percentage of the validation set, you have more data available for training your model. This can lead to a model that learns better from the available data, potentially improving its performance.
Cons:

Less Reliable Estimate: A smaller validation set may result in a less reliable estimate of your model's performance. The evaluation may be more sensitive to the specific examples in the validation set, making the estimate less stable. It might not give you a good indication of how well your model will generalize to unseen data.
In summary, increasing the percentage of the validation set generally leads to a more reliable estimate of performance but at the cost of having less data for training. Reducing the percentage of the validation set provides more training data but may result in a less reliable estimate. The choice of the validation set size depends on the specific goals of your machine learning project, the size and quality of your dataset, and your computational resources. Techniques like cross-validation can help balance this trade-off by repeatedly splitting the data into training and validation subsets to obtain a more robust performance estimate.







3.Answer

The choice of the percentage to reserve for the validation set depends on various factors, including the size of your dataset, the complexity of your model, and your specific goals. Balancing the trade-off between a reliable performance estimate and having sufficient data for training is essential. There is no one-size-fits-all answer, but here are some general guidelines:

80/20 Split: A common starting point is an 80/20 or 70/30 train/validation split, where 80% or 70% of the data is used for training, and the remaining 20% or 30% is used for validation. This provides a reasonable balance between having enough data for training and obtaining a relatively stable performance estimate.

Cross-Validation: Instead of a fixed train/validation split, you can use k-fold cross-validation. In k-fold cross-validation, the data is divided into k subsets (folds), and the model is trained and validated k times, each time using a different fold as the validation set. This approach ensures that every data point is used for both training and validation, and the results are averaged. Common choices for k include 5 or 10.

Stratified Sampling: If your dataset has class imbalance (i.e., some classes are rare), consider using stratified sampling to ensure that each class is represented proportionally in both the training and validation sets. This can be particularly important in classification tasks.

Data Size Consideration: If you have a very large dataset, you can allocate a smaller percentage to the validation set, and it will still contain a substantial number of examples. Conversely, if you have a small dataset, you might need to allocate a larger percentage to the validation set to obtain a representative sample.

Experimentation: Ultimately, it often comes down to experimentation. You can try different train/validation splits and cross-validation strategies and observe how they affect your model's performance. The choice may also depend on whether you prioritize obtaining the most accurate estimate of model performance or maximizing the amount of data available for training.

In summary, there is no universally optimal percentage for the validation set. The choice should be driven by the characteristics of your data and the goals of your project. It's often helpful to try different splits and validation strategies and assess their impact on your specific machine learning task.






#Multiple Splits

1.Answer

Yes, averaging the validation accuracy across multiple splits, as done in techniques like k-fold cross-validation, typically gives more consistent and reliable results compared to a single fixed train/validation split. Here's why averaging is beneficial:

Reduced Variance: Averaging validation results over multiple splits reduces the impact of random variations in the data. In a single fixed split, the performance estimate might be influenced by the specific examples in the validation set. In cross-validation, you systematically vary the validation set, which helps in reducing this variance.

Better Generalization: Cross-validation provides a more comprehensive assessment of how well your model generalizes to different subsets of the data. It gives you a more complete picture of your model's performance across various scenarios, which is valuable for understanding its robustness.

More Stable Estimate: Averaging over multiple folds smooths out the evaluation process and provides a more stable estimate of your model's performance. This stability makes it easier to compare models or make decisions based on their performance.

Utilizes More Data: Cross-validation utilizes a larger portion of your data for both training and validation. In k-fold cross-validation, you train and validate the model k times, using different subsets of data each time. This means you effectively use (k-1)/k of your data for training in each fold, which can help improve your model's performance.

Less Biased: Averaging results from multiple splits reduces the potential for bias that might be introduced by a single, arbitrary train/validation split. It provides a more unbiased estimate of your model's performance.

While averaging over multiple splits is generally recommended for a more robust estimate of model performance, it does come at the cost of increased computational time, as you need to train and evaluate the model multiple times. However, the benefits in terms of reliability and stability often outweigh the additional computational cost, especially when you want to make informed decisions about model selection, hyperparameter tuning, or assessing the model's expected performance on unseen data.







2.Answer

Averaging validation accuracy across multiple splits, as done in techniques like k-fold cross-validation, does provide a more accurate estimate of the expected test accuracy compared to a single fixed train/validation split. Here's why it leads to a more accurate estimate:

Better Representation of Data Variability: Cross-validation exposes your model to different subsets of the data during each fold. This variation in the validation sets provides a more comprehensive view of how your model performs on different portions of the data. As a result, it accounts for the natural variability in the dataset, making the estimate more accurate.

Reduced Bias: Averaging results over multiple folds reduces the potential for bias that can be introduced by a single, arbitrary train/validation split. This means that your performance estimate is less likely to be influenced by peculiarities in one particular split.

Use of More Data for Training: Cross-validation uses a larger portion of your data for training in each fold. This allows your model to learn from a greater variety of data points, potentially leading to a more accurate model.

More Robust Hyperparameter Tuning: If you're using cross-validation for hyperparameter tuning, the averaged results can guide you toward more robust choices for hyperparameters that perform well across different data subsets. This results in a model that's likely to generalize better to unseen data.

However, it's important to note that while cross-validation provides a more accurate estimate of expected test accuracy, it's still an estimate.

3.Answer

The number of iterations (or folds) in cross-validation can have an impact on the estimate of model performance. The effect of the number of iterations on the estimate varies, and whether you get a better estimate with higher iterations depends on several factors:

More Iterations (Higher k):

Pros:

With more iterations (higher k), you obtain a more robust estimate of model performance. The results are averaged over a larger number of validation sets, reducing the impact of randomness and variability in the data.
A higher number of iterations provides a better representation of how well the model generalizes to different subsets of the data, leading to a more comprehensive assessment.
Cons:

It increases computational cost. Running cross-validation with a higher number of iterations requires training and evaluating the model multiple times, which can be time-consuming, especially for large datasets or complex models.
There may be diminishing returns. Increasing k beyond a certain point might not significantly improve the estimate, especially if the dataset is large and diverse.
Fewer Iterations (Lower k):

Pros:

Reduced computational cost. Fewer iterations require less time and computational resources.
In some cases, a lower number of iterations might be sufficient to obtain a reasonably accurate estimate, especially if the dataset is large and the model has a consistent performance across different subsets.
Cons:

The estimate might be less reliable. With fewer iterations, the estimate of model performance may be more sensitive to the specific validation sets used in each fold, leading to a less stable estimate.
The estimate may not capture the full variability in the data. If the dataset has diverse characteristics, a lower number of iterations may not adequately represent how well the model generalizes to different scenarios.
In practice, the choice of the number of iterations (k) in cross-validation is often a trade-off between obtaining a more reliable estimate and managing computational resources. Common choices for k include 5-fold and 10-fold cross-validation, but these can be adjusted based on the specific dataset and computational constraints.

It's important to strike a balance that allows you to obtain a reasonably accurate estimate of model performance while not excessively increasing the time and resources required. Experimenting with different values of k and assessing their impact on your specific machine learning task is often the best approach to determine the optimal number of iterations for cross-validation.







4.Answer

Increasing the number of iterations (folds) in cross-validation can help mitigate the challenges associated with a very small train dataset or validation dataset to some extent, but it may not completely solve the problem. Here's how it affects the handling of small datasets:

Dealing with a Very Small Training Dataset:

If you have a very small training dataset, increasing the number of iterations can help by allowing the model to train on different subsets of the limited data. This can potentially improve the robustness of the model and provide a better estimate of its performance.
Dealing with a Very Small Validation Dataset:

Increasing the number of iterations does not directly address the issue of a very small validation dataset. If the validation dataset is small, each fold of cross-validation will still have a small validation set, which can lead to less reliable performance estimates. Increasing the number of iterations won't magically make the validation set larger.
Trade-offs and Considerations:

While increasing the number of iterations is beneficial for obtaining more robust performance estimates, there are trade-offs to consider:
Computational Cost: More iterations require more time and computational resources.
Diminishing Returns: There may be diminishing returns in terms of performance estimate improvement beyond a certain number of iterations, especially if the dataset is very small.
Alternative Approaches:

If you have an extremely small dataset, it's essential to consider alternative approaches:
Data Augmentation: You can apply data augmentation techniques to artificially increase the size of your training dataset by creating variations of existing data points.
Transfer Learning: Consider using pre-trained models or knowledge transfer from related tasks to improve model performance with limited data.
Simpler Models: With a very small dataset, complex models might overfit. Consider using simpler models with fewer parameters.
Regularization: Apply strong regularization techniques to prevent overfitting in the presence of limited training data.
In summary, while increasing the number of iterations in cross-validation can help with a small training dataset to some extent, it doesn't directly address the challenge of a small validation dataset. Dealing with extremely limited data often requires a combination of strategies, including data augmentation, model selection, regularization, and, if possible, acquiring more data to improve the robustness of your machine learning model.




